In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append the parent path into sys-path so wen can import necessary modules
sys.path.append('../')

In [4]:
from BackGround_modules.Class_1_Make_fourier_imgs import Make_Fourier
from BackGround_modules.Class_2_Classify_Fourier_Img import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

### Step_0_Define_Basic_Parameters

In [5]:
# define the year range
year_start = [f'{i}-01-01' for i in range(1990,2020,3)]
year_end   = [f'{i}-12-31' for i in range(1992,2020,3)]

year_range = list(zip(year_start,year_end))

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

# define the asset path for exporting
asset_path = 'users/wang8052664/Cloud_Free_Img'

In [6]:
year_range

[('1990-01-01', '1992-12-31'),
 ('1993-01-01', '1995-12-31'),
 ('1996-01-01', '1998-12-31'),
 ('1999-01-01', '2001-12-31'),
 ('2002-01-01', '2004-12-31'),
 ('2005-01-01', '2007-12-31'),
 ('2008-01-01', '2010-12-31'),
 ('2011-01-01', '2013-12-31'),
 ('2014-01-01', '2016-12-31'),
 ('2017-01-01', '2019-12-31')]

### Step_1: Create Landsat cloud-free images

In [7]:
for span in year_range[::-1]:
    
    # find the end year to determine the 
    start = span[0]
    end   = span[1]
    
    split = '-'
    span_name = f'{start.split(split)[0]}_{end.split(split)[0]}'

    
    # define the name for exporting
    export_name = f'Landsat_cloud_free_{span_name}'
    
    
    # find the right Landsat RAW data set
    if int(end.split(split)[0]) <= 2010:
        Landsat_img = ee.ImageCollection("LANDSAT/LT05/C01/T1")
        
    elif int(end.split(split)[0]) <= 2013:
        Landsat_img = ee.ImageCollection("LANDSAT/LE07/C01/T1")
        
    else:
        Landsat_img = ee.ImageCollection("LANDSAT/LC08/C01/T1")
        
    # Create a function to compute the min of the input mask to mask
    # pixels where any band is masked.
    def min_mask (img):
        return img.updateMask(img.mask().reduce('min'))
    
    # create the cloud-free img using SimpleComposite
    Filtered_img = Landsat_img.filterBounds(North_China_Plain)\
                              .filterDate(start,end)\
                              .map(lambda img: min_mask(img))
    
    cloud_free = ee.Algorithms.Landsat.simpleComposite(Filtered_img)\
                              .clip(North_China_Plain)
        
    # export
#     task = ee.batch.Export.image.toAsset(   image       = cloud_free,
#                                             description = f'Exporting {export_name}',
#                                             assetId     = f'{asset_path}/{export_name}',
#                                             region      = North_China_Plain.geometry(),
#                                             scale       = 30,
#                                             maxPixels   = int(1e13))
#     task.start()
    
    # print out the process
    print(f'Exporting {export_name}')

Exporting Landsat_cloud_free_2017_2019
Exporting Landsat_cloud_free_2014_2016
Exporting Landsat_cloud_free_2011_2013
Exporting Landsat_cloud_free_2008_2010
Exporting Landsat_cloud_free_2005_2007
Exporting Landsat_cloud_free_2002_2004
Exporting Landsat_cloud_free_1999_2001
Exporting Landsat_cloud_free_1996_1998
Exporting Landsat_cloud_free_1993_1995
Exporting Landsat_cloud_free_1990_1992


### Step_2: Create Sentinel cloud-free Image

In [7]:
# Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask  = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask)\
                 .select("B.*")\
                 .copyProperties(image, ["system:time_start"])

In [12]:
# here composite Sentinel-2 multispectrum image of year [2015-2017] for classification of [2014-2016]
#            and Sentinel-2 multispectrum image of year [2018-2019] for classification of [2017-2019]
# because composition of S2 image at year [2014-2016] are not fully cloud-free

Sentinel_year_range = [("2018-01-01","2019-12-31"),("2015-01-01","2017-12-31")]

Sentinel_img =  [ee.ImageCollection("COPERNICUS/S2")\
                      .filterBounds(North_China_Plain)\
                      .filterDate(*S_t)\
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                      .map(lambda x: maskS2clouds(x))\
                      .median()
                      .clip(North_China_Plain.geometry()) for S_t in Sentinel_year_range] 

# add 'Sentinel' to each band name of Sentinel img
Sentinel_img = [img.rename([f'Sentinel_{band}' for band in  img.bandNames().getInfo()]) for img in Sentinel_img]

In [ ]:
# Export the Sentinel_free image

for name,img in zip(['Sentinel_cloud_free_2018_2019','Sentinel_cloud_free_2015_2017'],
                    Sentinel_img):

    # export
    task = ee.batch.Export.image.toAsset(   image       = img,
                                            description = f'Exporting {name}',
                                            assetId     = f'{asset_path}/{name}',
                                            region      = North_China_Plain.geometry(),
                                            scale       = 10,
                                            maxPixels   = int(1e13))
    
    # start task
    task.start()
    
    # print out the process
    print(f'Exporting {name}')